In [10]:
import abc
import six
import numpy as np
import math
import random
import abc
from pygmo import hypervolume
from copy import deepcopy

In [1]:
class MulticriteriaObj:
    def __init__(self, dim, func_dim=2,lb=None, ub=None, minimizing=True, func_name=None):
        
        # define input range [lb,ub]
        if type(lb) is int or type(lb) is float or type(lb) is np.float:
            self.lowerbounds = np.array([lb for _ in range(dim)])
        elif type(lb) is list or type(lb) is np.array:
            self.lowerbounds = np.array(lb)
        else:
            print("type(lb) = %s"%(type(lb)))
            raise ValueError("Acceptable Lowerbounds: int, float, np.float, list, np.array.")
        
        if type(ub) is int or type(ub) is float or type(ub) is np.float:
            self.upperbounds = np.array([ ub for _ in range(dim)])
        elif type(ub) is list or type(ub) is np.array:
            self.upperbounds = np.array(ub)
        else:
            print("type(ub) = %s"%(type(ub)))            
            raise ValueError("Acceptable Upperbounds: int, float, np.float, list, np.array.")

        # define other parameters
        self.functiondim = func_dim
        self.name = func_name
        self.dimension = dim
        

    def get_function_dim(self):
        return self.functiondim
 
    def get_ref_point(self,ref_point):
        function_dim = self.get_function_dim()
        if self.minimizing == True:
            return [ref_point]*function_dim
        else:
            return [-ref_point]*function_dim
            
    def evaluate(self, x):       
        raise NotImplementedError("No evaluation function")

        
    def MinMax_evaluate(self, x):  
        if self.minimizing:
            return -1 * self.evaluate(x)
        else:
            return self.evaluate(x)

In [12]:

# multi-objectives

class MCMcCourt01(MulticriteriaObj):
    def __init__(self, dim, func_dim=2,lb=None, ub=None, minimizing=True, func_name=None):

        assert dim == 2
        func_dim = 2
        self.minimizing = True

        if func_name is None:
            func_name = "MCMcCourt01(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [-1 for _ in range(dim)]
            ub = [1 for _ in range(dim)]

        super().__init__(dim, func_dim=func_dim, lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)

    def MCMcCourt01_f1(self, x):
        x = np.array(x)
        y1 = (x[0] - .5) ** 2 + (x[1] + .4) ** 2
        return y1

    def MCMcCourt01_f2(self, x):
        x = np.array(x)
        y2 = -(np.sin(2 * x[0]) / (2 * x[0]) if abs(x[0]) > 1e-13 else 1.0) - np.exp(-2 * (x[1] - .2) ** 2)
        return y2

    def evaluate(self, x):
        return np.array([self.MCMcCourt01_f1(x), self.MCMcCourt01_f2(x)])


class MCMcCourt02(MulticriteriaObj):
    def __init__(self, dim, func_dim=2,lb=None, ub=None, minimizing=True, func_name=None):
        assert dim == 2
        func_dim = 2
        self.minlocation = np.array([[0.60065187, -0.49739315], [-1, -1]])
        self.minimizing = True

        if func_name is None:
            func_name = "MCMcCourt02(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [-1 for _ in range(dim)]
            ub = [1 for _ in range(dim)]

        super().__init__(dim, func_dim=func_dim,lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)


    def MCMcCourt02_f1(self, x):
        x = np.array(x)
        x1 = x[0]
        x2 = x[1]
        y1 = -(
                .5 * np.exp(-10 * ((x1 + .8) ** 2 + (x2 + .6) ** 2)) +
                .5 * np.exp(-9 * ((x1 + .7) ** 2 + (x2 - .4) ** 2)) +
                .6 * np.exp(-11 * ((x1 - .6) ** 2 + (x2 + .5) ** 2)) +
                .6 * np.exp(-8 * ((x1 - .8) ** 2 + (x2 - .3) ** 2))
        )
        return y1

    def MCMcCourt02_f2(self, x):
        x = np.array(x)
        y2 = -np.linalg.norm(x - self.minlocation[0])
        return y2

    def evaluate(self, x):
        return np.array([self.MCMcCourt02_f1(x), self.MCMcCourt02_f2(x)])


class MCMcCourt03(MulticriteriaObj):
    def __init__(self, dim, func_dim=2,lb=None, ub=None, minimizing=True, func_name=None):
        assert dim == 2
        func_dim = 2
        self.minimizing = True
        self.minlocation = np.array([[-0.71086527, 0.34990005], [1, -1]])

        if func_name is None:
            func_name = "MCMcCourt03(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [-1 for _ in range(dim)]
            ub = [1 for _ in range(dim)]

        super().__init__(dim, func_dim=func_dim, lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)

    def MCMcCourt03_f1(self, x):
        x = np.array(x)
        x1 = x[0]
        x2 = x[1]
        y1 = -(
                .6 * np.exp(-10 * ((x1 + .8) ** 2 + .3 * (x2 + .6) ** 2)) +
                .6 * np.exp(-9 * (.4 * (x1 + .7) ** 2 + .4 * (x2 - .4) ** 2)) +
                .6 * np.exp(-11 * (.2 * (x1 - .6) ** 2 + .5 * (x2 + .5) ** 2)) +
                .6 * np.exp(-8 * (.3 * (x1 - .8) ** 2 + .6 * (x2 - .3) ** 2))
        )
        return y1

    def MCMcCourt03_f2(self, x):
        x = np.array(x)
        y2 = -np.linalg.norm(x - self.minlocation[0])
        return y2

    def evaluate(self, x):
        return np.array([self.MCMcCourt03_f1(x), self.MCMcCourt03_f2(x)])


class MCMcCourt04(MulticriteriaObj):
    def __init__(self, dim, func_dim=2,lb=None, ub=None, minimizing=True, func_name=None):
        assert dim == 2
        func_dim = 2
        self.minimizing = True

        if func_name is None:
            func_name = "MCMcCourt04(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [-1 for _ in range(dim)]
            ub = [1 for _ in range(dim)]

        super().__init__(dim, func_dim=func_dim, lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)

    def MCMcCourt04_f1(self, x):
        x = np.array(x)
        x1 = x[0]
        x2 = x[1]
        y1 = -(
                .5 * np.exp(-10 * ((x1 + .8) ** 2 + .3 * (x2 + .6) ** 2)) +
                .5 * np.exp(-9 * (.4 * (x1 + .7) ** 2 + .4 * (x2 - .4) ** 2)) +
                .5 * np.exp(-11 * (.2 * (x1 - .6) ** 2 + .5 * (x2 + .5) ** 2)) +
                .5 * np.exp(-11 * (.6 * (x1) ** 2 + .5 * (x2 + .8) ** 2)) +
                .5 * np.exp(-12 * (.4 * (x1 - .1) ** 2 + .7 * (x2 - .8) ** 2)) +
                .5 * np.exp(-13 * (.8 * (x1) ** 2 + .7 * (x2) ** 2)) +
                .5 * np.exp(-8 * (.3 * (x1 - .8) ** 2 + .6 * (x2 - .3) ** 2))
        )
        return y1

    def MCMcCourt04_f2(self, x):
        x = np.array(x)
        y2 = -np.linalg.norm(x - self.minlocation[0])
        return y2

    def evaluate(self, x):
        return np.array([self.MCMcCourt04_f1(x), self.MCMcCourt04_f2(x)])


class MCMcCourt05(MulticriteriaObj):
    def __init__(self, dim, func_dim=2,lb=None, ub=None, minimizing=True, func_name=None):
        assert dim == 2
        func_dim = 2
        self.minimizing = True

        if func_name is None:
            func_name = "MCMcCourt05(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [-1 for _ in range(dim)]
            ub = [1 for _ in range(dim)]

        super().__init__(dim, func_dim=func_dim, lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)

    def MCMcCourt05_f1(self, x):
        x = np.array(x)
        x1 = x[0]
        x2 = x[1]
        y1 = -(
                .5 * np.exp(-10 * ((x1 + .8) ** 2 + .3 * (x2 + .6) ** 2)) +
                .5 * np.exp(-9 * (.4 * (x1 + .7) ** 2 + .4 * (x2 - .4) ** 2)) +
                .5 * np.exp(-11 * (.2 * (x1 - .6) ** 2 + .5 * (x2 + .5) ** 2)) +
                .5 * np.exp(-11 * (.6 * (x1) ** 2 + .5 * (x2 + .8) ** 2)) +
                .5 * np.exp(-12 * (.4 * (x1 - .1) ** 2 + .7 * (x2 - .8) ** 2)) +
                .5 * np.exp(-13 * (.8 * (x1) ** 2 + .7 * (x2) ** 2)) +
                .5 * np.exp(-8 * (.3 * (x1 - .8) ** 2 + .6 * (x2 - .3) ** 2))
        )
        return y1

    def MCMcCourt05_f2(self, x):
        x = np.array(x)
        sol1 = self.minlocation[0]
        sol2 = np.array([[0.08827771, -0.6818113]])
        y2 = -1 / (1 / np.linalg.norm(x - sol1) + 1 / np.linalg.norm(x - sol2))
        return y2

    def evaluate(self, x):
        return np.array([self.MCMcCourt05_f1(x), self.MCMcCourt05_f2(x)])


class MCMcCourt06(MulticriteriaObj):
    def __init__(self, dim, func_dim=2,lb=None, ub=None, minimizing=True, func_name=None):
        assert dim == 3
        func_dim = 2
        self.minimizing = True

        if func_name is None:
            func_name = "MCMcCourt06(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [-1 for _ in range(dim)]
            ub = [1 for _ in range(dim)]

        super().__init__(dim, func_dim=func_dim, lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)

    def MCMcCourt06_f1(self, x):
        x = np.array(x)
        x1 = x[0]
        x2 = x[1]
        x3 = x[2]
        y1 = -(
                .5 * np.exp(-10 * ((x1 + .8) ** 2 + .3 * (x2 + .6) ** 2 + .7 * (x3 + .7) ** 2)) +
                .5 * np.exp(-9 * (.4 * (x1 + .7) ** 2 + .4 * (x2 - .4) ** 2 + .4 * (x3 - .1) ** 2)) +
                .5 * np.exp(-11 * (.2 * (x1 - .6) ** 2 + .5 * (x2 + .5) ** 2 + .6 * (x3 + .2) ** 2)) +
                .5 * np.exp(-11 * (.6 * (x1) ** 2 + .5 * (x2 + .8) ** 2 + .3 * (x3 - .8) ** 2)) +
                .5 * np.exp(-12 * (.4 * (x1 - .1) ** 2 + .7 * (x2 - .8) ** 2 + .8 * (x3 - .4) ** 2)) +
                .5 * np.exp(-13 * (.8 * (x1) ** 2 + .7 * (x2) ** 2 + .5 * (x3) ** 2)) +
                .5 * np.exp(-8 * (.3 * (x1 - .8) ** 2 + .6 * (x2 - .9) ** 2 + .3 * (x3 + .6) ** 2)) +
                .5 * np.exp(-8 * (.5 * (x1 - .3) ** 2 + .5 * (x2 + .2) ** 2 + .5 * (x3 - .4) ** 2)) +
                .5 * np.exp(-8 * (.5 * (x1 + .4) ** 2 + .4 * (x2 - .3) ** 2 + .7 * (x3 + .3) ** 2)) +
                .5 * np.exp(-8 * (.8 * (x1 + .2) ** 2 + .5 * (x2 + .6) ** 2 + .7 * (x3 - .9) ** 2)) +
                .5 * np.exp(-8 * (.9 * (x1 - .5) ** 2 + .6 * (x2 - .7) ** 2 + .4 * (x3 + .1) ** 2))
        )
        return y1

    def MCMcCourt06_f2(self, x):
        x = np.array(x)
        y2 = -np.linalg.norm(x - self.minlocation[0])
        return y2

    def evaluate(self, x):
        return np.array([self.MCMcCourt06_f1(x), self.MCMcCourt06_f2(x)])


class MCMcCourt07(MulticriteriaObj):
    def __init__(self, dim, func_dim=2,lb=None, ub=None, minimizing=True, func_name=None):
        assert dim == 3
        func_dim = 2
        self.minimizing = True

        if func_name is None:
            func_name = "MCMcCourt07(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [-1 for _ in range(dim)]
            ub = [1 for _ in range(dim)]

        super().__init__(dim, func_dim=func_dim, lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)

    def MCMcCourt07_f1(self, x):
        x = np.array(x)
        x1 = x[0]
        x2 = x[1]
        x3 = x[2]
        y1 = -(
                .5 * np.exp(-10 * ((x1 + .8) ** 2 + .3 * (x2 + .6) ** 2 + .7 * (x3 + .7) ** 2)) +
                .5 * np.exp(-9 * (.4 * (x1 + .7) ** 2 + .4 * (x2 - .4) ** 2 + .4 * (x3 - .1) ** 2)) +
                .5 * np.exp(-11 * (.2 * (x1 - .6) ** 2 + .5 * (x2 + .5) ** 2 + .6 * (x3 + .2) ** 2)) +
                .5 * np.exp(-11 * (.6 * (x1) ** 2 + .5 * (x2 + .8) ** 2 + .3 * (x3 - .8) ** 2)) +
                .5 * np.exp(-12 * (.4 * (x1 - .1) ** 2 + .7 * (x2 - .8) ** 2 + .8 * (x3 - .4) ** 2)) +
                .5 * np.exp(-13 * (.8 * (x1) ** 2 + .7 * (x2) ** 2 + .5 * (x3) ** 2)) +
                .5 * np.exp(-8 * (.3 * (x1 - .8) ** 2 + .6 * (x2 - .9) ** 2 + .3 * (x3 + .6) ** 2)) +
                .5 * np.exp(-8 * (.5 * (x1 - .3) ** 2 + .5 * (x2 + .2) ** 2 + .5 * (x3 - .4) ** 2)) +
                .5 * np.exp(-8 * (.5 * (x1 + .4) ** 2 + .4 * (x2 - .3) ** 2 + .7 * (x3 + .3) ** 2)) +
                .5 * np.exp(-8 * (.8 * (x1 + .2) ** 2 + .5 * (x2 + .6) ** 2 + .7 * (x3 - .9) ** 2)) +
                .5 * np.exp(-8 * (.9 * (x1 - .5) ** 2 + .6 * (x2 - .7) ** 2 + .4 * (x3 + .1) ** 2))
        )
        return y1

    def MCMcCourt07_f2(self, x):
        x = np.array(x)
        sol1 = self.minlocation[0]
        sol2 = np.array([[-0.02179115, 0.01597782, -0.01427473]])
        y2 = -1 / (1 / np.linalg.norm(x - sol1) + 1 / np.linalg.norm(x - sol2))
        return y2

    def evaluate(self, x):
        return np.array([self.MCMcCourt07_f1(x), self.MCMcCourt07_f2(x)])


class MCMcCourt08(MulticriteriaObj):
    def __init__(self, dim, func_dim=2,lb=None, ub=None, minimizing=True, func_name=None):
        assert dim == 12
        self.input_dim = dim
        func_dim = 2
        self.minimizing = True
        if func_name is None:
            func_name = "MCMcCourt08(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [700, 200, 30, 50, 18, 5, 51, 30, 0, 175, 0, 5]
            ub = [1277, 1277, 175, 140, 80, 58, 140, 60, 58, 210, 12, 35]

        super().__init__(dim, func_dim=func_dim, lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)

    def Cal_sx(self, x):
        x = np.array(x)
        sx = np.empty(self.input_dim)
        for k in range(self.input_dim):
            sx[k] = (x[k] - self.lowerbounds[k]) / (self.upperbounds[k] - self.lowerbounds[k])
        return sx

    def MCMcCourt08_f1(self, x):
        sx = self.Cal_sx(x)
        nonconvex_factor = 3
        scaling1 = np.linspace(1 / self.input_dim, 1.0, self.input_dim)
        scaling2 = np.linspace(1.3, 1 / self.input_dim, self.input_dim)
        distance1_sq = np.dot(sx - 1 / 3, scaling1 * (sx - 1 / 3))
        distance2_sq = np.dot(sx - 2 / 3, scaling2 * (sx - 2 / 3))
        y1 = (
                -np.exp(-2 * distance1_sq * nonconvex_factor) -
                1.5 * np.exp(-1.6 * distance2_sq * nonconvex_factor)
        )
        return y1

    def MCMcCourt08_f2(self, x):
        sx = self.Cal_sx(x)
        nonconvex_factor = 3
        scaling1 = np.linspace(1.3, 1 / self.input_dim, self.input_dim)
        scaling2 = np.linspace(1 / self.input_dim, 1.0, self.input_dim)
        distance1_sq = np.dot(sx - 0.25, scaling1 * (sx - 0.25))
        distance2_sq = np.dot(sx - 0.6, scaling2 * (sx - 0.6))
        y2 = (
                -1.6 * np.exp(-1.2 * distance1_sq * nonconvex_factor) -
                .8 * np.exp(-1.7 * distance2_sq * nonconvex_factor)
        )
        return y2

    def evaluate(self, x):
        return np.array([self.MCMcCourt08_f1(x), self.MCMcCourt08_f2(x)])



class Poloni(MulticriteriaObj):
    def __init__(self, dim, func_dim=2,lb=None, ub=None, minimizing=True, func_name=None):
        assert dim == 2
        func_dim = 2
        self.minimizing = True

        if func_name is None:
            func_name = "Poloni(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [-np.pi for _ in range(dim)]
            ub = [np.pi for _ in range(dim)]

        super().__init__(dim, func_dim=func_dim, lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)

    def Poloni_f1(self, x):
        x = np.array(x)
        x, y = x[0], x[1]
        A1 = .5 * np.sin(1) - 2 * np.cos(1) + np.sin(2) - 1.5 * np.cos(2)
        A2 = 1.5 * np.sin(1) - np.cos(1) + 2 * np.sin(2) - .5 * np.cos(2)
        B1 = .5 * np.sin(x) - 2 * np.cos(x) + np.sin(y) - 1.5 * np.cos(y)
        B2 = 1.5 * np.sin(x) - np.cos(x) + 2 * np.sin(y) - .5 * np.cos(y)
        return 1 + (A1 - B1) ** 2 + (A2 - B2) ** 2

    def Poloni_f2(self, x):
        x = np.array(x)
        x, y = x[0], x[1]
        return (x + 3) ** 2 + (y + 1) ** 2

    def evaluate(self, x):
        return np.array([self.Poloni_f1(x), self.Poloni_f2(x)])


class ZitzlerDebThiele01(MulticriteriaObj):
    def __init__(self, dim, func_dim=2,lb=None, ub=None, minimizing=True, func_name=None):
        assert dim == 4
        func_dim = 2
        self.minimizing = True

        if func_name is None:
            func_name = "ZitzlerDebThiele01(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [0 for _ in range(dim)]
            ub = [1 for _ in range(dim)]

        super().__init__(dim, func_dim=func_dim, lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)


    def ZitzlerDebThiele01_f1(self, x):
        x = np.array(x)
        return x[0]

    def ZitzlerDebThiele01_f2(self, x):
        x = np.array(x)
        g = 1 + 9.0 / 29 * sum(x[1:])
        return g * (1 - np.sqrt(x[0] / g))

    def evaluate(self, x):
        return np.array([self.ZitzlerDebThiele01_f1(x), self.ZitzlerDebThiele01_f2(x)])


class ZitzlerDebThiele02(MulticriteriaObj):
    def __init__(self, dim, func_dim=2,lb=None, ub=None, minimizing=True, func_name=None):
        assert dim == 3
        func_dim = 2
        self.minimizing = True

        if func_name is None:
            func_name = "ZitzlerDebThiele02(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [0 for _ in range(dim)]
            ub = [1 for _ in range(dim)]

        super().__init__(dim, func_dim=func_dim, lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)


    def ZitzlerDebThiele02_f1(self, x):
        x = np.array(x)
        return x[0]

    def ZitzlerDebThiele02_f2(self, x):
        x = np.array(x)
        g = 1 + 9.0 / 29 * sum(x[1:])
        return g * (1 - (x[0] / g) ** 2)

    def evaluate(self, x):
        return np.array([self.ZitzlerDebThiele02_f1(x), self.ZitzlerDebThiele02_f2(x)])


class ZitzlerDebThiele03(MulticriteriaObj):
    def __init__(self, dim, func_dim=2,lb=None, ub=None, minimizing=True, func_name=None):
        assert dim == 4
        func_dim = 2
        self.minimizing = True

        if func_name is None:
            func_name = "ZitzlerDebThiele03(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [0 for _ in range(dim)]
            ub = [1 for _ in range(dim)]


        super().__init__(dim, func_dim=func_dim, lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)

    def ZitzlerDebThiele03_f1(self, x):
        x = np.array(x)
        return x[0]

    def ZitzlerDebThiele03_f2(self, x):
        x = np.array(x)
        g = 1 + 9.0 / 29 * sum(x[1:])
        return g * (1 - np.sqrt(x[0] / g) - (x[0] / g) * np.sin(10 * np.pi * x[0]))

    def evaluate(self, x):
        return np.array([self.ZitzlerDebThiele03_f1(x), self.ZitzlerDebThiele03_f2(x)])



# Reference: http://www.egr.msu.edu/~kdeb/papers/k2006001.pdf
class ZitzlerDebThiele04(MulticriteriaObj):
    def __init__(self, dim, func_dim=2,lb=None, ub=None, minimizing=True, func_name=None):
        assert dim == 3
        self.input_dim = dim
        func_dim = 2
        self.minimizing = True

        if func_name is None:
            func_name = "ZitzlerDebThiele04(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [0, -5, -5]
            ub = [1, 5, 5]

        super().__init__(dim, func_dim=func_dim,lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)


    def evaluate(self, x):
        x = np.array(x)
        D = np.array([  # Randomly constructed
            [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, -0.66, -0.11, -0.61, -0.19, -0.00, -0.65, 0.99, -0.86, 0.06],
            [0, -0.35, 0.55, -0.79, -0.05, -0.46, 0.15, -0.03, -0.83, -0.22],
            [0, 0.39, -0.70, -0.15, -0.56, -0.17, 0.23, -0.78, -0.06, -0.13],
            [0, -0.86, -0.77, -0.87, -0.14, 0.03, -0.96, 0.91, 0.13, -0.81],
            [0, 0.04, 0.09, 0.55, -0.04, -0.83, 0.12, 0.52, -0.67, -0.40],
            [0, -0.06, -0.94, 0.67, 0.68, 0.61, 0.43, 0.34, -0.08, 0.46],
            [0, 0.33, -0.40, -0.83, -0.64, 0.67, 0.49, 0.08, 0.56, 0.42],
            [0, 0.54, -0.77, -0.25, -0.52, -0.71, -0.36, -0.44, 0.40, -0.98],
            [0, -0.87, -0.00, 0.36, -0.38, 0.84, 0.72, 0.94, 0.42, -0.64],
        ])
        y = np.dot(D[:self.input_dim, :self.input_dim], x)
        f1 = y[0] ** 2
        g = 1 + 10 * (self.input_dim - 1) + sum(y[1:] ** 2 - 10 * np.cos(4 * np.pi * y[1:]))
        f2 = g * (1 - np.sqrt(f1 / g))
        return np.array([f1, f2])


class ZitzlerDebThiele06(MulticriteriaObj):
    def __init__(self, dim, func_dim=2,lb=None, ub=None, minimizing=True, func_name=None):
        assert dim == 3
        self.input_dim = dim
        func_dim = 2
        self.minimizing = True

        if func_name is None:
            func_name = "ZitzlerDebThiele06(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [0 for _ in range(dim)]
            ub = [1 for _ in range(dim)]


        super().__init__(dim, func_dim=func_dim, lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)

    def ZitzlerDebThiele06_f1(self, x):
        x = np.array(x)
        f1 = 1 - np.exp(-4 * x[0]) * np.sin(6 * np.pi * x[0]) ** 6
        return f1

    def ZitzlerDebThiele06_f2(self, x):
        x = np.array(x)
        f1 = 1 - np.exp(-4 * x[0]) * np.sin(6 * np.pi * x[0]) ** 6
        g = 1 + 9.0 * (sum(x[1:]) / (self.input_dim - 1)) ** .25
        return g * (1 - (f1 / g) ** 2)

    def evaluate(self, x):
        return np.array([self.ZitzlerDebThiele06_f1(x), self.ZitzlerDebThiele06_f2(x)])


class BraninCurrin(MulticriteriaObj):
    def __init__(self, dim, func_dim=2,lb=None, ub=None, minimizing=True, func_name=None):
        assert dim == 2
        func_dim = 2
        self.minimizing = True
        self.ref_point = [500.0, 500.0]  # ref:botorch
        if func_name is None:
            func_name = "BraninCurrin(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [0 for _ in range(dim)]
            ub = [1 for _ in range(dim)]
        super().__init__(dim, func_dim=func_dim, lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)

    def Currin(self, x):
        x = np.array(x)
        x1, x2 = x
        return float(((1 - math.exp(-0.5 * (1 / x[1]))) * ((2300 * pow(x[0], 3) + 1900 * x[0] * x[0] + 2092 * x[0] + 60) / (100 * pow(x[0], 3) + 500 * x[0] * x[0] + 4 * x[0] + 20))))

    def Branin(self, x):
        x = np.array(x)
        x = deepcopy(x)
        x[0] = 15 * x[0] - 5
        x[1] = 15 * x[1]
        return float(np.square(x[1] - (5.1 / (4 * np.square(math.pi))) * np.square(x[0]) + (5 / math.pi) * x[0] - 6) + 10 * (1 - (1. / (8 * math.pi))) * np.cos(x[0]) + 10)

    def evaluate(self, x):
        return np.array([self.Currin(x), self.Branin(x)])


class DTLZ1(MulticriteriaObj):
    def __init__(self, dim, func_dim=2,lb=None, ub=None, minimizing=True, func_name=None):
        assert dim >= func_dim - 1
        self.input_dim = dim
        self.func_dim = 4
        self.minimizing = True
        self.ref_point = [400.0] * self.func_dim  # ref: botorch
        if func_name is None:
            func_name = "DTLZ1(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [0 for _ in range(dim)]
            ub = [1 for _ in range(dim)]
        super().__init__(dim, func_dim=func_dim, lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)


    def DTLZ1_f1(self, x):
        x = np.array(x)
        d = self.input_dim
        k = self.input_dim - self.func_dim + 1
        g = 0
        for i in range(d):
            g = g + pow(x[i] - 0.5, 2) - math.cos(20 * math.pi * (x[i] - 0.5))
        g = 100 * (k + g)
        y1 = (1 + g) * 0.5 * x[0] * x[1] * x[2]
        return y1

    def DTLZ1_f2(self, x):
        x = np.array(x)
        d = self.input_dim
        k = self.input_dim - self.func_dim + 1
        g = 0
        for i in range(d):
            g = g + pow(x[i] - 0.5, 2) - math.cos(20 * math.pi * (x[i] - 0.5))
        g = 100 * (k + g)
        y2 = (1 + g) * 0.5 * (1 - x[2]) * x[0] * x[1]
        return y2

    def DTLZ1_f3(self, x):
        x = np.array(x)
        d = self.input_dim
        k = self.input_dim - self.func_dim + 1
        g = 0
        for i in range(d):
            g = g + pow(x[i] - 0.5, 2) - math.cos(20 * math.pi * (x[i] - 0.5))
        g = 100 * (k + g)
        y3 = (1 + g) * 0.5 * (1 - x[1]) * x[0]
        return y3

    def DTLZ1_f4(self, x):
        x = np.array(x)
        d = self.input_dim
        k = self.input_dim - self.func_dim + 1
        g = 0
        for i in range(d):
            g = g + pow(x[i] - 0.5, 2) - math.cos(20 * math.pi * (x[i] - 0.5))
        g = 100 * (k + g)
        y4 = (1 + g) * 0.5 * (1 - x[0])
        return y4

    def evaluate(self, x):
        return np.array([self.DTLZ1_f1(x), self.DTLZ1_f2(x), self.DTLZ1_f3(x), self.DTLZ1_f4(x)])


class DTLZ2(MulticriteriaObj):
    def __init__(self, dim, func_dim=2,lb=None, ub=None, minimizing=True, func_name=None):
        assert dim >= func_dim - 1
        self.input_dim = dim
        self.func_dim = 4
        self.minimizing = True
        if func_name is None:
            func_name = "DTLZ2(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [0 for _ in range(dim)]
            ub = [1 for _ in range(dim)]
        super().__init__(dim, func_dim=func_dim, lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)


    def DTLZ2_f1(self, x):
        x = np.array(x)
        d = self.input_dim
        g = 0
        for i in range(d):
            g = g + pow(x[i] - 0.5, 2)
        y1 = (1 + g) * math.cos(math.pi * x[0] / 2) * math.cos(math.pi * x[1] / 2) * math.cos(math.pi * x[2] / 2)
        return y1

    def DTLZ2_f2(self, x):
        x = np.array(x)
        d = self.input_dim
        g = 0
        for i in range(d):
            g = g + pow(x[i] - 0.5, 2)
        y2 = (1 + g) * math.cos(math.pi * x[0] / 2) * math.cos(math.pi * x[1] / 2) * math.sin(math.pi * x[2] / 2)
        return y2

    def DTLZ2_f3(self, x):
        x = np.array(x)
        d = self.input_dim
        g = 0
        for i in range(d):
            g = g + pow(x[i] - 0.5, 2)
        y3 = (1 + g) * math.cos(math.pi * x[0] / 2) * math.sin(math.pi * x[1] / 2)
        return y3

    def DTLZ2_f4(self, x):
        x = np.array(x)
        d = self.input_dim
        g = 0
        for i in range(d):
            g = g + pow(x[i] - 0.5, 2)
        y4 = (1 + g) * math.sin(math.pi * x[0] / 2)
        return y4

    def evaluate(self, x):
        return np.array([self.DTLZ2_f1(x), self.DTLZ2_f2(x), self.DTLZ2_f3(x), self.DTLZ2_f4(x)])


class DTLZ3(MulticriteriaObj):
    def __init__(self, dim, func_dim=2,lb=None, ub=None, minimizing=True, func_name=None):
        assert dim >= func_dim - 1
        self.input_dim = dim
        self.func_dim = 5
        self.minimizing = True
        if func_name is None:
            func_name = "DTLZ3(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [0 for _ in range(dim)]
            ub = [1 for _ in range(dim)]
        super().__init__(dim, func_dim=func_dim, lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)


    def DTLZ3_f1(self, x):
        x = np.array(x)
        d = self.input_dim
        k = self.input_dim - self.func_dim + 1
        g = 0
        for i in range(d):
            g = g + pow(x[i] - 0.5, 2) - math.cos(20 * math.pi * (x[i] - 0.5))
        g = 100 * (k + g)
        y1 = (1 + g) * math.cos(math.pi * x[0] / 2) * math.cos(math.pi * x[1] / 2) * math.cos(math.pi * x[2] / 2) * math.cos(math.pi * x[3] / 2)
        return y1

    def DTLZ3_f2(self, x):
        x = np.array(x)
        d = self.input_dim
        k = self.input_dim - self.func_dim + 1
        g = 0
        for i in range(d):
            g = g + pow(x[i] - 0.5, 2) - math.cos(20 * math.pi * (x[i] - 0.5))
        g = 100 * (k + g)
        y2 = (1 + g) * math.cos(math.pi * x[0] / 2) * math.cos(math.pi * x[1] / 2) * math.cos(math.pi * x[2] / 2) * math.sin(math.pi * x[3] / 2)
        return y2

    def DTLZ3_f3(self, x):
        x = np.array(x)
        d = self.input_dim
        k = self.input_dim - self.func_dim + 1
        g = 0
        for i in range(d):
            g = g + pow(x[i] - 0.5, 2) - math.cos(20 * math.pi * (x[i] - 0.5))
        g = 100 * (k + g)
        y3 = (1 + g) * math.cos(math.pi * x[0] / 2) * math.cos(math.pi * x[1] / 2) * math.sin(math.pi * x[2] / 2)
        return y3

    def DTLZ3_f4(self, x):
        x = np.array(x)
        d = self.input_dim
        k = self.input_dim - self.func_dim + 1
        g = 0
        for i in range(d):
            g = g + pow(x[i] - 0.5, 2) - math.cos(20 * math.pi * (x[i] - 0.5))
        g = 100 * (k + g)
        y4 = (1 + g) * math.cos(math.pi * x[0] / 2) * math.sin(math.pi * x[1] / 2)
        return y4

    def DTLZ3_f5(self, x):
        x = np.array(x)
        d = self.input_dim
        k = self.input_dim - self.func_dim + 1
        g = 0
        for i in range(d):
            g = g + pow(x[i] - 0.5, 2) - math.cos(20 * math.pi * (x[i] - 0.5))
        g = 100 * (k + g)
        y5 = (1 + g) * math.sin(math.pi * x[0] / 2)
        return y5

    def evaluate(self, x):
        return np.array([self.DTLZ3_f1(x), self.DTLZ3_f2(x), self.DTLZ3_f3(x), self.DTLZ3_f4(x), self.DTLZ3_f5(x)])

# reference: http://www.sfu.ca/~ssurjano
class SRDSDZ(MulticriteriaObj):
    def __init__(self, dim, func_dim=6,lb=None, ub=None, minimizing=True, func_name=None):
        assert dim == 2
        self.input_dim = dim
        self.func_dim = func_dim
        self.minimizing = True
        if func_name is None:
            func_name = "SRDSDZ(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [-5.12 for _ in range(dim)]
            ub = [5.12 for _ in range(dim)]
        super().__init__(dim, func_dim=func_dim, lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)
        
    # http://www.sfu.ca/~ssurjano/spheref.html
    def Sphere(self, x):
        x = np.array(x)
        y1 = sum(x**2)
        return y1
    
    #http://www.sfu.ca/~ssurjano/rastr.html
    def Rastrigin(self, x):
        x = np.array(x)
        d = self.input_dim
        y2 = 10*d
        for xi in x:
            y2 += xi**2-10*math.cos(2*np.pi*xi)
        return y2
    
    #https://www.sfu.ca/~ssurjano/drop.html
    def DropWave(self, x):
        x = np.array(x)
        d = self.input_dim
        tmp = sum(x**2)
        y3 = -(1+math.cos(12*math.sqrt(tmp)))/(0.5*tmp+2)
        return y3

    #http://www.sfu.ca/~ssurjano/sumsqu.html
    def sum_squares(self, x):
        x = np.array(x)
        y4 = 0
        for i in range(len(x)):
            y4 += (i+1)*(x[i]**2)
        return y4
    
    # http://www.sfu.ca/~ssurjano/dixonpr.html
    def Dixon_price(self, x):
        x = np.array(x)
        y5 = (x[0]-1)**2+2*(2*x[1]**2-x[0])**2
        return y5
    
    # http://www.sfu.ca/~ssurjano/zakharov.html
    def Zakharov(self, x):
        x = np.array(x)
        sum1 = sum(x**2)
        sum2 = 0.5*1*x[0]+0.5*2*x[1]
        y6 = sum1 +sum2**2 + sum2**4
        return y6

    def evaluate(self, x):
        return np.array([self.Sphere(x), self.Rastrigin(x), self.DropWave(x), self.sum_squares(x), self.Dixon_price(x),self.Zakharov(x)])

class SRDS(MulticriteriaObj):
    def __init__(self, dim, func_dim=4,lb=None, ub=None, minimizing=True, func_name=None):
        assert dim == 2
        self.input_dim = dim
        self.func_dim = func_dim
        self.minimizing = True
        if func_name is None:
            func_name = "SRDS(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [-5.12 for _ in range(dim)]
            ub = [5.12 for _ in range(dim)]
        super().__init__(dim, func_dim=func_dim, lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)
        
    # http://www.sfu.ca/~ssurjano/spheref.html
    def Sphere(self, x):
        x = np.array(x)
        y1 = sum(x**2)
        return y1
    
    #http://www.sfu.ca/~ssurjano/rastr.html
    def Rastrigin(self, x):
        x = np.array(x)
        d = self.input_dim
        y2 = 10*d
        for xi in x:
            y2 += xi**2-10*math.cos(2*np.pi*xi)
        return y2
    
    #https://www.sfu.ca/~ssurjano/drop.html
    def DropWave(self, x):
        x = np.array(x)
        d = self.input_dim
        tmp = sum(x**2)
        y3 = -(1+math.cos(12*math.sqrt(tmp)))/(0.5*tmp+2)
        return y3

    #http://www.sfu.ca/~ssurjano/sumsqu.html
    def sum_squares(self, x):
        x = np.array(x)
        y4 = 0
        for i in range(len(x)):
            y4 += (i+1)*(x[i]**2)
        return y4
    
    # http://www.sfu.ca/~ssurjano/dixonpr.html
    def Dixon_price(self, x):
        x = np.array(x)
        y5 = (x[0]-1)**2+2*(2*x[1]**2-x[0])**2
        return y5
    
    # http://www.sfu.ca/~ssurjano/zakharov.html
    def Zakharov(self, x):
        x = np.array(x)
        sum1 = sum(x**2)
        sum2 = 0.5*1*x[0]+0.5*2*x[1]
        y6 = sum1 +sum2**2 + sum2**4
        return y6

    def evaluate(self, x):
        return np.array([self.Sphere(x), self.Rastrigin(x), self.DropWave(x), self.sum_squares(x)])

class SRD(MulticriteriaObj):
    def __init__(self, dim, func_dim=3,lb=None, ub=None, minimizing=True, func_name=None):
        assert dim == 2
        self.input_dim = dim
        self.func_dim = func_dim
        self.minimizing = True
        if func_name is None:
            func_name = "SRDS(D%d" % dim + "K%d)" % func_dim
        if lb is None or ub is None:
            lb = [-5.12 for _ in range(dim)]
            ub = [5.12 for _ in range(dim)]
        super().__init__(dim, func_dim=func_dim, lb=lb, ub=ub, minimizing=minimizing, func_name=func_name)
        
    # http://www.sfu.ca/~ssurjano/spheref.html
    def Sphere(self, x):
        x = np.array(x)
        y1=2
        #y1 = sum(x**2)
        return y1
    
    #http://www.sfu.ca/~ssurjano/rastr.html
    def Rastrigin(self, x):
        x = np.array(x)
        d = self.input_dim
        y2 = 10*d
        for xi in x:
            y2 += xi**2-10*math.cos(2*np.pi*xi)
        return y2
    
    #https://www.sfu.ca/~ssurjano/drop.html
    def DropWave(self, x):
        x = np.array(x)
        d = self.input_dim
        tmp = sum(x**2)
        y3 = -(1+math.cos(12*math.sqrt(tmp)))/(0.5*tmp+2)
        return y3

    #http://www.sfu.ca/~ssurjano/sumsqu.html
    def sum_squares(self, x):
        x = np.array(x)
        y4 = 0
        for i in range(len(x)):
            y4 += (i+1)*(x[i]**2)
        return y4
    
    # http://www.sfu.ca/~ssurjano/dixonpr.html
    def Dixon_price(self, x):
        x = np.array(x)
        y5 = (x[0]-1)**2+2*(2*x[1]**2-x[0])**2
        return y5
    
    # http://www.sfu.ca/~ssurjano/zakharov.html
    def Zakharov(self, x):
        x = np.array(x)
        sum1 = sum(x**2)
        sum2 = 0.5*1*x[0]+0.5*2*x[1]
        y6 = sum1 +sum2**2 + sum2**4
        return y6

    def evaluate(self, x):
        return np.array([self.Sphere(x), self.Rastrigin(x), self.DropWave(x)])
    